In [1]:
import pandas as pd
import numpy as np
import itertools as it

In [43]:
import seaborn as sns
import matplotlib as mp
from scipy.spatial.distance import cosine

In [3]:
pref = np.zeros((943, 1682), dtype = int)
movies = pd.read_csv('u.item', sep='|', header = None, encoding='latin-1')

In [4]:
len(movies)

1682

In [60]:
movies.iloc[9][1]

'Richard III (1995)'

In [5]:
file = open('u.data', 'r')
for line in file:
    col = line.split('\t')
    pref[int(col[0]) - 1][int(col[1]) - 1] = int(col[2])

In [6]:
data = []
f = open('u.data','r')
    

In [7]:
pref

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [8]:
pref.shape

(943, 1682)

In [9]:
a = np.where(pref == 5)

In [10]:
a

(array([  0,   0,   0, ..., 942, 942, 942]),
 array([  0,   5,   8, ..., 720, 927, 942]))

In [11]:
df = {}
for i in range(pref.shape[0]):
    df[i] = []
for i in range(len(a[0])):
    df[a[0][i]].append('__label__' + str(a[1][i]))

In [12]:
in1 = list(df.values())

In [14]:
canc = []
for i in range(len(in1)):
    if len(in1[i]) == 0:
        canc.append(i)

In [18]:
c = 0
for i in canc:
    in1.pop(i - c)
    c += 1

In [19]:
for i in in1:
    if len(i) == 0:
        print('r')

In [20]:
in2 = [' '.join(x) + '\n' for x in in1]

In [22]:
file1 = open("mltrain.txt","w") 
file1.writelines(in2)
file1.close()

In [23]:
import csv
d = []
emb = []
with open('/mnt/ml-100k/mlmodel.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        d.append(row[0])
        emb.append(np.array([float(x) for x in row[1:]]))

In [24]:
d1 = dict(zip(d,emb))

In [25]:
emb1 = list(d1.values())

In [37]:
useremb = {}
for i in range(len(in1)):
    c = np.zeros(100)
    for j in in1[i]:
        c += d1[j]
    c = c/len(in1[i])
    useremb[i] = c 

In [54]:
def reco(n):
    m = -0.9
    for k in useremb:
        if (1 - cosine(useremb[n],useremb[k]) > m) and (k != n):
            m = (1 - cosine(useremb[n],useremb[k]))
            user = k
    ans = []
    for i in range(len(pref[n])):
        if pref[n][i] == 0:
            if pref[user][i] != 0:
                ans.append((pref[user][i],i))
    return(ans)
        

In [61]:
[movies.iloc[x[1]][1] for x in sorted(reco(89), reverse = True) if x[0] == 5 ]

['Grease (1978)', 'Return of the Jedi (1983)', 'Ed Wood (1994)']